In [257]:
import backtrader as bt
import backtrader.talib as btalib
import pandas as pd
from datetime import datetime

file_path = './data/DOGEUSDT-1m-2023-10.csv'
doge_data = pd.read_csv(file_path)

doge_data['datetime'] = pd.to_datetime(doge_data['open_time'], unit='ms')
doge_data.set_index('datetime', inplace=True)

class PandasData(bt.feeds.PandasData):
    lines = ('datetime',)
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', None),
    )

In [258]:
class CombinedStrategy(bt.Strategy):
    params = (
        ('ema_period', 5),
        ('ma_period', 10),
        ('rsi_period', 14),
        ('rsi_low', 20),
        ('rsi_high', 80),
        ('boll_period', 20),
        ('macd1', 12),
        ('macd2', 26),
        ('macdsignal', 9),
        ('atr_period', 14),
        ('atr_multiplier', 3),
    )

    def __init__(self):
        self.ema = bt.indicators.ExponentialMovingAverage(self.data.close, period=self.params.ema_period)
        self.ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.ma_period)
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)
        self.boll = bt.indicators.BollingerBands(self.data.close, period=self.params.boll_period)
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd1, period_me2=self.params.macd2, period_signal=self.params.macdsignal)
        self.obv = btalib.OBV(self.data.close, self.data.volume)
        # self.atr = bt.indicators.ATR(self.data.high, self.data.low, self.data.close, period=self.params.atr_period)
        self.high = self.data.high
        self.low = self.data.low
        self.close = self.data.close
        # self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)
        self.atr = btalib.ATR(self.high, self.low, self.close, period=self.params.atr_period)
        # self.atr = btalib.ATR(self.data, period=self.params.atr_period)
        self.order = None
        self.buy_size = 1000
        self.buy_price = None
        self.stop_loss_price = None
        self.stop_loss = None
        self.trade_history = []

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
        log_line = f'{dt.isoformat()}, {txt}\n'

        with open('./output/trading_strategy.txt', 'a') as f:
            f.write(log_line)

    def log_trade(self, trade_type, price, size, value, cash):
        trade_datetime = self.datas[0].datetime.datetime(0)
        self.trade_history.append({
            'type': trade_type,
            'datetime': trade_datetime,
            'price': price,
            'size': size,
            'value': value,
            'cash_remaining': cash
        })

    def next(self):
        cash = self.broker.get_cash()
        value = self.broker.get_value()
        
        self.log(f'Close: {self.data.close[0]}, EMA: {self.ema[0]}, RSI: {self.rsi[0]}, Bollinger Mid: {self.boll.mid[0]}, OBV: {self.obv[0]}, MACD: {self.macd[0]}, ATR: {self.atr[0]}, Cash: {cash}')

        # Buy Logic
        if not self.position:
            if self.rsi < self.params.rsi_low and self.data.close[0] < self.ema[0] and self.obv[0] > self.obv[-1]:
                size = self.buy_size
                self.order = self.buy(size=size)
                self.buy_price = self.data.close[0]
                self.log('BUY EXECUTED')
                self.log_trade('buy', self.data.close[0], size, value, cash)
                # Set dynamic stop loss point
                self.stop_loss_price = self.buy_price - self.atr[0] * self.params.atr_multiplier

        # Check if stop loss is triggered
        if self.position and self.stop_loss_price is not None and self.data.close[0] < self.stop_loss_price:
            self.close()  # Sell all holdings if current price is below stop loss
            self.log('STOP LOSS TRIGGERED')
            self.stop_loss_price = None  # Reset stop loss point

        # Sell Logic
        elif self.position and self.rsi > self.params.rsi_high and self.data.close[0] > self.ema[0] and self.obv[0] < self.obv[-1]:
            self.close()  # Sell all holdings
            self.log('SELL EXECUTED')
            self.log_trade('sell', self.data.close[0], self.position.size, value, cash)
            self.buy_price = None
            self.stop_loss_price = None  # Clear stop loss point

    def stop(self):
        total_profit_loss = 0
        last_buy_price = None

        with open('./output/trade_history.txt', 'w') as f:
            for trade in self.trade_history:
                f.write(f"{trade}\n")

                if trade['type'] == 'buy':
                    last_buy_price = trade['price']
                elif trade['type'] == 'sell' and last_buy_price is not None:
                    profit_loss = (trade['price'] - last_buy_price) * trade['size']
                    total_profit_loss += profit_loss
                    last_buy_price = None 

        with open('./output/trading_strategy.txt', 'a') as f:
            f.write(f"Final Portfolio Value: {self.broker.getvalue()}\n")
            f.write(f"Total PnL: {total_profit_loss}\n")

        self.total_profit_loss = total_profit_loss

In [259]:
import numpy as np

cerebro = bt.Cerebro()
cerebro.addstrategy(CombinedStrategy)

data_feed = bt.feeds.PandasData(dataname=doge_data)

# Adding data and setting initial cash
cerebro.adddata(data_feed)
cerebro.broker.set_cash(1000)

backtest_result = cerebro.run()

# Extracting the results
final_strategy = backtest_result[0]
final_portfolio_value = final_strategy.broker.getvalue()
total_pnl = final_strategy.total_profit_loss

print(f"Date", doge_data.index[-1])
print(f"Final Portfolio Value: {final_portfolio_value}")
print(f"Total PnL: {total_pnl}")

# Calculate the log returns
doge_data['log_return'] = np.log(doge_data['close'] / doge_data['close'].shift(1))

# Calculate the standard deviation of log returns
std_dev = doge_data['log_return'].std()

minutes_in_year = 525600
annualized_volatility = std_dev * np.sqrt(minutes_in_year)

print(f"Annualized Volatility: {annualized_volatility}")
print(f"Sharpe Ratio: {total_pnl / annualized_volatility}")
print(f"Total Trades: {len(final_strategy.trade_history)}")
print(f"Total Turnover: {doge_data['volume'].sum()}")

Date 2023-10-31 23:59:00
Final Portfolio Value: 1005.8199999999999
Total PnL: 0.43999999999999595
Annualized Volatility: 0.5501930294627371
Sharpe Ratio: 0.7997193283776342
Total Trades: 32
Total Turnover: 20585750037.0


In [260]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(30, 15), dpi=600)

plt.plot(prices.index, prices, label='Price', linewidth=0.5)  # Adjust the linewidth here

# Add markers for buy and sell signals
for trade in final_strategy.trade_history:
    if trade['type'] == 'buy':
        plt.scatter(trade['datetime'], trade['price'], marker='^', color='green', s=10)
    elif trade['type'] == 'sell':
        plt.scatter(trade['datetime'], trade['price'], marker='v', color='red', s=10)

plt.title('DOGE Price with Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator())
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


NameError: name 'prices' is not defined

<Figure size 18000x9000 with 0 Axes>